📌 Что такое ColumnTransformer
ColumnTransformer — это «конвейер внутри конвейера». Он говорит:
«Вот тебе таблица X. Бери числовые колонки и пропускай их через StandardScaler, а категориальные — через OneHotEncoder. Потом склеивай результаты в одну матрицу».
Так мы избегаем ручного df[...], .fillna(), .get_dummies() и т.д. — всё происходит в одном объекте, который можно класть в Pipeline и обучать вместе с моделью.

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_openml
X, y = fetch_openml('wine-quality-red', as_frame=True, return_X_y=True)

In [ ]:
X

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify =y)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
numeric = X.select_dtypes('number').columns

In [ ]:
categorical = X.select_dtypes('category').columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical)
    ])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline(steps=[
    ('prep', preprocessor),
    ('clf', RandomForestClassifier(random_state=42))
])

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'clf__n_estimators': [100, 300],
    'clf__max_depth':    [None, 10, 20]
}
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid.fit(X_train, y_train)
print('Лучшие параметры:', grid.best_params_)
print('F1 на тесте:', grid.score(X_test, y_test))

Лучшие параметры: {'clf__max_depth': None, 'clf__n_estimators': 300}
F1 на тесте: 0.4106503568164593


Сохранение и открытие модели

In [ ]:
import joblib
joblib.dump(grid.best_estimator_, 'wine_rf_pipeline.pkl')

['wine_rf_pipeline.pkl']

In [ ]:
model = joblib.load("wine_rf_pipeline.pkl")

In [ ]:
model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index([], dtype='object'))])),
                ('clf',
                 RandomForestClassifier(n_estimators=300, random_state=42))])

Кратко и «на пальцах»:

1. GridSearchCV  
   Это «автоматический переборщик».  
   • Ты задаёшь список гиперпараметров (`clf__max_depth`, `clf__n_estimators`).  
   • GridSearchCV делает кросс-валидацию (например, 5-fold) для **каждой комбинации** этих значений.  
   • В итоге выбирает ту комбинацию, при которой скор (у нас F1) на валидации максимален.  
   • Возвращает объект, уже обученный на лучших параметрах.

2. Pipeline  
   Это «конвейер» = упорядоченная цепочка шагов.  
   • На вход подаётся сырой DataFrame.  
   • Каждый шаг последовательно преобразует данные (ColumnTransformer → StandardScaler/OneHot → модель).  
   • Всё вместе ведёт себя как одна «большая» модель: можно `.fit()`, `.predict()`, `.score()` и прогонять в GridSearchCV, не боясь «утечки» данных.

Итого:  
Pipeline = **что делать с данными**, GridSearchCV = **какие цифры лучше подобрать для этого «что»**.

clf__n_estimators: сколько деревьев построить в RandomForest.
[100, 300] → проверим 100 и 300 деревьев.

clf__max_depth: максимальная глубина каждого дерева.
[None, 10, 20] → проверим «не ограничено», 10 и 20 уровней.

f1 - точность

F₁ = 2·(precision·recall)/(precision+recall).
Чем ближе к 1 — тем лучше модель предсказывает классы.

Коротко и по делу:

1. Взяли датасет «Wine Quality-Red» (1599 образцов, 11 признаков + целевая переменная quality).  
2. Разбили на train/test (stratify=quality).  
3. Через `ColumnTransformer` автоматически нормализовали числовые признаки (`StandardScaler`), а категориальные (если бы были) – закодировали OneHot.  
4. Внутри `Pipeline` обучили `RandomForestClassifier` и подобрали 2 гиперпараметра `GridSearchCV`.  
5. Лучшая комбинация: `max_depth=None`, `n_estimators=300`.  
6. **Результат**: F1-макро ≈ 0.41 на тесте. Это средний показатель: модель работает лучше случайного, но есть куда расти (признаковая инженерия, баланс классов, другие алгоритмы).

Итог работы:
`GridSearchCV` берёт **весь Pipeline** (preprocessor → дерево), прокручивает его 6 раз (по 5-fold кроссу каждый) и выбирает ту комбинацию `n_estimators` + `max_depth`, при которой среднее F1 на валидации максимально.